In [2]:
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as k
import numpy as np
from keras.preprocessing import image

In [3]:
img_width, img_height = 150, 150

train_data_dir = 'dataset/train'
valid_data_dir = 'dataset/valid'
nb_train_samples = 140
nb_validation_samples = 20
epochs = 50
batch_size = 5

In [4]:
if k.image_data_format=='Channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [5]:
train_datagen = ImageDataGenerator(
    rescale= 1. / 255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True
)

test_datagen = ImageDataGenerator(rescale=1./255)

In [6]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size = (img_width, img_height),
    batch_size = batch_size,
    class_mode = 'binary'
)

validation_generator = test_datagen.flow_from_directory(
    valid_data_dir,
    target_size = (img_width, img_height),
    batch_size = batch_size,
    class_mode = 'binary'
)

Found 280 images belonging to 2 classes.
Found 40 images belonging to 2 classes.


In [7]:
#Create Model
model= Sequential()
model.add( Conv2D(32, (3,3), input_shape=input_shape))
model.add( Activation('relu'))
model.add( MaxPooling2D( pool_size=(2,2)))

model.summary()

model.add( Conv2D(32, (3,3)))
model.add( Activation('relu'))
model.add( MaxPooling2D( pool_size=(2,2)))

model.add( Conv2D(64, (3,3)))
model.add( Activation('relu'))
model.add( MaxPooling2D( pool_size=(2,2)))

model.add( Flatten())
model.add( Dense(64))
model.add( Activation('relu'))
model.add( Dropout(0.5))
model.add( Dense(1))
model.add( Activation('sigmoid'))

model.summary()

model.compile(loss= 'binary_crossentropy',
              optimizer= 'rmsprop',
              metrics=['accuracy'])


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      896       
_________________________________________________________________
activation (Activation)      (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 32)        0         
Total params: 896
Trainable params: 896
Non-trainable params: 0
_________________________________________________________________
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      896       
_________________________________________________________________
activation (Activation)      (None, 148, 148, 32)      0         
______________________________________

In [12]:
#this is the augmentation configuration we will use for training
model.fit_generator(
    train_generator,
    steps_per_epoch= nb_train_samples // batch_size,
    epochs=epochs,
    validation_data= validation_generator,
    validation_steps= nb_validation_samples // batch_size
    )

C:\Users\Soumya Sekhar\anaconda3\lib\site-packages\keras\engine\training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
Epoch 1/50
28/28 [==============================] - 38s 1s/step - loss: 0.3716 - accuracy: 0.8500 - val_loss: 0.3886 - val_accuracy: 0.8500
Epoch 2/50
28/28 [==============================] - 25s 871ms/step - loss: 0.3917 - accuracy: 0.8000 - val_loss: 0.8182 - val_accuracy: 0.8000
Epoch 3/50
28/28 [==============================] - 20s 722ms/step - loss: 0.4755 - accuracy: 0.8000 - val_loss: 0.2578 - val_accuracy: 0.8500
Epoch 4/50
28/28 [==============================] - 23s 818ms/step - loss: 0.3942 - accuracy: 0.8500 - val_loss: 0.3717 - val_accuracy: 0.9000
Epoch 5/50
28/28 [==============================] - 23s 820ms/step - loss: 0.4494 - accuracy: 0.7857 - val_loss: 0.6686 - val_accuracy: 0.8000
Epoch 

In [31]:
def predict(path_of_img):
    img_pred= image.load_img(path_of_img, target_size= (150,150))
    img_pred= image.img_to_array(img_pred)
    img_pred= np.expand_dims( img_pred, axis=0)

    result = model.predict(img_pred)
    if result[0][0]==1:
        predictn = "The above image is watermarked."
    else:
        predictn = "The above image is not watermarked."
        
    print(result)
    print(predictn)

    import cv2
    img = cv2.imread(path_of_img)
    img = cv2.resize(img, (300, 300))
    cv2.imshow('image', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


In [43]:
predict('dataset/valid/watermarked/100.jpeg')

[[1.]]
The above image is watermarked.
